<a href="https://colab.research.google.com/github/Zywia/AD-projekt-2-/blob/additional_data/projekt_2_AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert-for-tf2

In [2]:
!pip install tf-models-nightly

In [9]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import pickle
import pandas as pd
# from bert.tokenization
from bert import bert_tokenization
import official.nlp.bert.tokenization
# from bert.tokenization import FullSentencePieceTokenizer


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/1",
                            trainable=True)

In [6]:
train_data, test_data = tfds.load(name="civil_comments", split=["train", "test"], data_dir="/content/drive/My Drive/AD/civil_comments", batch_size=-1, as_supervised=True)



In [7]:
train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

In [10]:
more_to_train_negative = pd.read_csv("more.csv")
more_to_test_negative = pd.read_csv("test_n.csv")

In [11]:
len( more_to_test_negative.comment_text.to_list() )

6229

In [12]:
negative_comments_positions = np.argwhere(train_labels == 1)[:,0]
negative_comments = train_examples[negative_comments_positions]
negative_comments = np.hstack((negative_comments,  more_to_train_negative.comment_text.to_numpy() ) )


negative_comments_positions_test = np.argwhere(test_labels == 1)[:,0]

negative_comments_test = test_examples[negative_comments_positions_test]
negative_comments_test = np.hstack((negative_comments_test,  more_to_test_negative.comment_text.to_numpy() ) )





In [13]:

not_all_positive_comments_positions = np.argwhere(train_labels == 0)[:negative_comments.shape[0], 0]
not_all_positive_comments_positions_test = np.argwhere(test_labels == 0)[:negative_comments_test.shape[0], 0]

not_all_positive_comments = train_examples[not_all_positive_comments_positions]
not_all_positive_comments_test = test_examples[not_all_positive_comments_positions_test]


print( not_all_positive_comments_positions_test.shape)
print( not_all_positive_comments_positions.shape)

(6451,)
(20577,)


In [14]:
labels_T = np.hstack( (np.ones(negative_comments.shape[0]), np.zeros(negative_comments.shape[0]) ) )
labels_t = np.hstack( (np.ones(negative_comments_test.shape[0]), np.zeros(negative_comments_test.shape[0]) ) )


all_to_train = np.hstack((negative_comments, not_all_positive_comments ))
all_to_test = np.hstack(( negative_comments_test, not_all_positive_comments_test))



perm_T = np.random.permutation(all_to_train.shape[0])
perm_t = np.random.permutation(all_to_test.shape[0])

train_examples = all_to_train[perm_T]
train_labels = labels_T[perm_T]

test_examples = all_to_test[perm_t]
test_labels = labels_t[perm_t]

In [15]:
idx = 41

print (test_labels[idx] )
print (test_examples[idx])

1.0
I see you morons have removed my comments about the US Fiat currency being measured against other Fiat currency instead of goods and gold/silver. Why don't you IDIOTS learn how economics really works and that the mainstream indicators are bullshit.


In [16]:
?official.nlp.bert.tokenization.FullSentencePieceTokenizer

In [17]:
sp_model_file = bert_layer.resolved_object.sp_model_file.asset_path.numpy()
tokenizer = official.nlp.bert.tokenization.FullSentencePieceTokenizer(sp_model_file)


def tokenize_text(txt):
    tokenized_txt = tokenizer.tokenize(txt)
    tokenized_txt = ["[CLS]"] + tokenized_txt + ["[SEP]"]
    tokenized_txt = tokenizer.convert_tokens_to_ids(tokenized_txt)
    return tokenized_txt

In [18]:

train_examples = list(map(tokenize_text, train_examples))
test_examples = list(map(tokenize_text, test_examples))

In [19]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(train_examples, maxlen=100, padding='post')
max_seq_length = x_train.shape[1]
x_test = tf.keras.preprocessing.sequence.pad_sequences(test_examples, maxlen=max_seq_length, padding='post', truncating='post')

train_mask = (x_train != 0).astype(int)
test_mask = (x_test != 0).astype(int)

train_segments = np.zeros(x_train.shape)
test_segments = np.zeros(x_test.shape)

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")

bert_inputs = [input_word_ids, input_mask, segment_ids]
pooled_output, sequence_output = bert_layer(bert_inputs)

final_layer = tf.keras.layers.Dense(1, activation="sigmoid")
final_output = final_layer(pooled_output)
# tf.keras.layers.


In [20]:
model = tf.keras.models.Model(inputs=bert_inputs, outputs=final_output)

In [21]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              # optimizer=tf.optimizers.Adam(learning_rate=1e-4), 
              optimizer=tf.optimizers.Adam(), 

              metrics=["accuracy"])

In [22]:
model.fit([x_train, train_mask, train_segments], train_labels, epochs=2, batch_size=32,
          validation_data=([x_test, test_mask, test_segments], test_labels))

Epoch 1/2
 113/1287 [=>............................] - ETA: 20:55 - loss: 0.7252 - accuracy: 0.5003

KeyboardInterrupt: ignored

In [23]:
import pandas as pd

text = pd.read_csv("notI.csv", names=["sentance"], index_col=False)
text = text.to_numpy()

print (text[:,0].tolist())
text = text[:,0].tolist()
text = list(map(tokenize_text, text))
print(text[0])
text = tf.keras.preprocessing.sequence.pad_sequences(text, maxlen=100, padding='post')
mask = (text != 0).astype(int)
segments = np.zeros(text.shape)


['temporrytext that is absoulutlely not important', 'anotehre aboulutally not important text', 'you are ugly so ugly', 'hate speech']
[2, 14935, 139, 622, 11969, 30, 25, 12178, 111, 8331, 5866, 102, 52, 681, 3]


In [24]:
model([text, mask, segments])

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.5538093],
       [0.5538093],
       [0.5538093],
       [0.5538093]], dtype=float32)>

In [30]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 11683584    input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [29]:
m = tf.keras.models.load_model("/content/drive/My Drive/AD/check/my_model")
print (m([text, mask, segments]) )
m.summary()


tf.Tensor(
[[0.00341263]
 [0.00178851]
 [0.996042  ]
 [0.89366364]], shape=(4, 1), dtype=float32)
Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 11683584    input_word_ids[0][0]   

In [26]:
resoult = m([text, mask, segments])

In [27]:
resoult = resoult > 0.5
resoult = np.array(resoult, dtype='int')

In [78]:
np.savetxt("foo.csv", resoult, delimiter=",")

In [55]:
model.save("/content/drive/My Drive/AD/check/my_model2")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /content/drive/My Drive/AD/check/my_model/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/AD/check/my_model/assets
